# 신경망 학습 실습 - MNIST

이번 실습에서는 다음과 같은 과정을 직접 구현해보면서 앞서 배운 내용을 복습해보도록 하겠습니다.  
  
> **신경망 학습이란?**  
> 신경망에는 가중치와 편향이 있고 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라고 합니다.

신경망 학습은 다음과 같은 절차에 따라 진행됩니다.
1. 미니배치 - 훈련 데이터 중 일부를 무작위로 가져옵니다. 이때 무작위로 가져온 데이터를 **미니배치**라 합니다.  
   
2. 기울기 산출 - 미니배치의 손실 함수의 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구합니다.  
   
3. 매개변수 갱신 - 가중치 매개변수 기울기 방향으로 지정한 학습률 만큼 갱신합니다.  
   
4. 1~3단계 반복

## Library import

In [2]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

## 2층 신경망 구현
2층 신경망을 구현해보도록 하겠습니다.
- 입력층의 크기(input_size) : 784
- 은닉층의 크기(hidden_size) : 50
- 출력층의 크기(output_size) : 10
- 가중치 초기화 분산(weight_init_std) : 0.01

In [6]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        self.params={}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self,x):
        W1,W2 = self.params['W1'],self.params['W2']
        b1,b2 = self.params['b1'],self.params['b2']

        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2) + b2
        y = softmax(a2)

        return y
    
    def loss(self,x,t):
        y = self.predict(x)

        return cross_entropy_error(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)

        accuracy = np.sum(y==t) / float(x.shape[0])

        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W : self.loss(x,t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W,self.params['b2'])

        return grads

가중치의 형상을 확인해보면서 2층 신경망이 올바르게 구현 됬는지 확인해보겠습니다.

In [8]:
net = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
print("|W1|=",net.params['W1'].shape)
print("|b1|=",net.params['b1'].shape)
print("|W2|=",net.params['W2'].shape)
print("|b2|=",net.params['b2'].shape)

|W1|= (784, 50)
|b1|= (50,)
|W2|= (50, 10)
|b2|= (10,)


입력한대로 올바르게 구현된 것을 확인 할 수 있습니다.

## 신경망 학습 구현
신경망 구조는 구현했으니(`TwoLayerNet`) 이제 신경망 절차에 따라 신경망 학습을 구현해보도록 하겠습니다.

In [ ]:
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 불러오기
(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)

# train 손실값을 담을 리스트 생성
train_loss_list=[]

# 하이퍼파라미터 값 설정
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

# 학습
for i in range(iters_num):
    # 1.미니배치 설정
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 2.기울기 산출
    grads = net.numerical_gradient(x_batch,t_batch)

    # 3.매개변수 갱신
    for key in ('W1','b1','W2','b2'):
        net.params[key] -= learning_rate * grads[key]

    # train_loss_list에 loss 추가 
    loss = net.loss(x_batch,t_batch)
    train_loss_list.append(loss)


신경망 학습 절차에 따라 신경망 학습을 구현했습니다. 하지만 아직 우리는 정확도를 측정하지 않았습니다. 모든 훈련 데이터를 돌았을 때 정확도를 산출하는 코드도 한번 구현해보도록 하겠습니다.

In [ ]:
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 불러오기
(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)

# train 손실값을 담을 리스트 생성
train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

# 하이퍼파라미터 값 설정
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
iter_per_epoch = max(iters_num / batch_size,1)

# 학습
for i in range(iters_num):
    # 1.미니배치 설정
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 2.기울기 산출
    grads = net.numerical_gradient(x_batch,t_batch)

    # 3.매개변수 갱신
    for key in ('W1','b1','W2','b2'):
        net.params[key] -= learning_rate * grads[key]

    # train_loss_list에 loss 추가 
    loss = net.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    # 정확도 산출
    if i % iter_per_epoch == 0:
        train_acc = net.accuracy(x_train,t_train)
        test_acc = net.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('Train Accuracy = {:.4f}'.format(train_acc))
        print('Test Accuracy = {:.4f}'.format(test_acc))

